In [314]:
## Cargar datos con colab
## =============================================================================

#from google.colab import drive
#import os

#drive.mount('/gdrive')

In [315]:
#os.chdir("/gdrive/MyDrive/Seminario_Ciencia_Datos_Jaramillo_Higuita")
#!ls

# Dataset

El dataset puede ser descargado desde este link, dentro de este notebook se usa la version CSV del mismo

https://www.datos.gov.co/Educaci-n/Saber-11-2019-2/ynam-yc42/about_data

NOTA: el archivo cambia de nombre con respecto a la fecha de descarga por lo cual sera necesario cambiar el nombre del archivo a la hora de cargar el data frame, el archivo se carga en una carpeta llamada "data" relativo a la ubicacion de este notebook

Resultados anonimizados de las pruebas de Saber 11 del segundo semestre del año 2019-2

# Instalaciones

In [316]:
!pip install prince
!pip install feature-engine

# Objetivo

Predecir los resultados de las pruebas a partir de los datos del estudiente y su familia, utilizando modelos de regresión

# Importación de librerias

In [317]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import prince
from scipy import stats
from feature_engine.imputation import CategoricalImputer
from sklearn import preprocessing
from feature_engine.creation import CyclicalFeatures

In [318]:
pd.set_option('display.max_columns', None)

# Carga del dataset

In [319]:
df = pd.read_csv("data/Saber_11__2019-2_20240510.csv", low_memory=False)
df.head(5)

,ESTU_TIPODOCUMENTO,ESTU_NACIONALIDAD,ESTU_GENERO,ESTU_FECHANACIMIENTO,PERIODO,ESTU_CONSECUTIVO,ESTU_ESTUDIANTE,ESTU_TIENEETNIA,ESTU_PAIS_RESIDE,ESTU_ETNIA,ESTU_DEPTO_RESIDE,ESTU_COD_RESIDE_DEPTO,ESTU_MCPIO_RESIDE,ESTU_COD_RESIDE_MCPIO,FAMI_ESTRATOVIVIENDA,FAMI_PERSONASHOGAR,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,FAMI_TRABAJOLABORPADRE,FAMI_TRABAJOLABORMADRE,FAMI_TIENEINTERNET,FAMI_TIENESERVICIOTV,FAMI_TIENECOMPUTADOR,FAMI_TIENELAVADORA,FAMI_TIENEHORNOMICROOGAS,FAMI_TIENEAUTOMOVIL,FAMI_TIENEMOTOCICLETA,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_NUMLIBROS,FAMI_COMELECHEDERIVADOS,FAMI_COMECARNEPESCADOHUEVO,FAMI_COMECEREALFRUTOSLEGUMBRE,FAMI_SITUACIONECONOMICA,ESTU_DEDICACIONLECTURADIARIA,ESTU_DEDICACIONINTERNET,ESTU_HORASSEMANATRABAJA,ESTU_TIPOREMUNERACION,COLE_CODIGO_ICFES,COLE_COD_DANE_ESTABLECIMIENTO,COLE_NOMBRE_ESTABLECIMIENTO,COLE_GENERO,COLE_NATURALEZA,COLE_CALENDARIO,COLE_BILINGUE,COLE_CARACTER,COLE_COD_DANE_SEDE,COLE_NOMBRE_SEDE,COLE_SEDE_PRINCIPAL,COLE_AREA_UBICACION,COLE_JORNADA,COLE_COD_MCPIO_UBICACION,COLE_MCPIO_UBICACION,COLE_COD_DEPTO_UBICACION,COLE_DEPTO_UBICACION,ESTU_PRIVADO_LIBERTAD,ESTU_COD_MCPIO_PRESENTACION,ESTU_MCPIO_PRESENTACION,ESTU_DEPTO_PRESENTACION,ESTU_COD_DEPTO_PRESENTACION,PUNT_LECTURA_CRITICA,PERCENTIL_LECTURA_CRITICA,DESEMP_LECTURA_CRITICA,PUNT_MATEMATICAS,PERCENTIL_MATEMATICAS,DESEMP_MATEMATICAS,PUNT_C_NATURALES,PERCENTIL_C_NATURALES,DESEMP_C_NATURALES,PUNT_SOCIALES_CIUDADANAS,PERCENTIL_SOCIALES_CIUDADANAS,DESEMP_SOCIALES_CIUDADANAS,PUNT_INGLES,PERCENTIL_INGLES,DESEMP_INGLES,PUNT_GLOBAL,PERCENTIL_GLOBAL,ESTU_INSE_INDIVIDUAL,ESTU_NSE_INDIVIDUAL,ESTU_NSE_ESTABLECIMIENTO,ESTU_ESTADOINVESTIGACION,ESTU_GENERACION-E
0,TI,COLOMBIA,M,02/18/2002 12:00:00 AM,20194,SB11201940031558,ESTUDIANTE,No,COLOMBIA,-,MAGDALENA,47,SANTA ANA,47707,Estrato 3,5 a 6,Cinco,Educación profesional completa,Secundaria (Bachillerato) completa,No aplica,"Trabaja en el hogar, no trabaja o estudia",Si,Si,No,Si,Si,No,No,No,MÁS DE 100 LIBROS,1 o 2 veces por semana,3 a 5 veces por semana,Nunca o rara vez comemos eso,Peor,Entre 30 y 60 minutos,Entre 30 y 60 minutos,Menos de 10 horas,"Si, en efectivo",39008,147707000156,INSTITUCION EDUCATIVA DEPARTAMENTAL ANTONIO BR...,MIXTO,OFICIAL,A,N,TÉCNICO/ACADÉMICO,147707000156,COL DPTAL ANTONIO BRUGES CARMONA,S,URBANO,MAÑANA,47707,SANTA ANA,47,MAGDALENA,N,47707,SANTA ANA,MAGDALENA,47,47,33,2,48,42,2,37,15,1,30,8,1,54.0,67,A1,208,25,50.9421557404314,2,2.0,PUBLICAR,NO
1,TI,COLOMBIA,M,06/28/2003 12:00:00 AM,20194,SB11201940303339,ESTUDIANTE,No,COLOMBIA,-,BOGOTÁ,11,BOGOTÁ D.C.,11001,Estrato 3,9 o más,Tres,Técnica o tecnológica completa,Técnica o tecnológica completa,Trabaja por cuenta propia (por ejemplo plomero...,Es dueño de un negocio pequeño (tiene pocos em...,Si,Si,Si,Si,Si,Si,No,No,MÁS DE 100 LIBROS,Todos o casi todos los días,3 a 5 veces por semana,1 o 2 veces por semana,Mejor,Entre 30 y 60 minutos,Entre 30 y 60 minutos,Menos de 10 horas,"Si, en efectivo",128611,311001105383,GIMN SAN ANGELO,MIXTO,NO OFICIAL,A,-,ACADÉMICO,311001105383,GIMN SAN ANGELO,S,URBANO,COMPLETA,11001,BOGOTÁ D.C.,11,BOGOTÁ,N,11001,BOGOTÁ D.C.,BOGOTÁ,11,60,76,3,65,88,3,54,71,2,59,83,3,63.0,88,A2,299,83,59.767864431576,3,4.0,PUBLICAR,NO
2,TI,COLOMBIA,M,06/02/2001 12:00:00 AM,20194,SB11201940255017,ESTUDIANTE,No,COLOMBIA,-,BOLIVAR,13,CARTAGENA DE INDIAS,13001,Estrato 1,5 a 6,Dos,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,Es vendedor o trabaja en atención al público,Es vendedor o trabaja en atención al público,No,Si,No,Si,No,No,No,No,26 A 100 LIBROS,Todos o casi todos los días,Todos o casi todos los días,3 a 5 veces por semana,Igual,Entre 30 y 60 minutos,Más de 3 horas,0,"Si, en efectivo",35493,113001001719,INSTITUCION EDUCATIVA PROMOCION SOCIAL DE C/GENA.,MIXTO,OFICIAL,A,N,TÉCNICO/ACADÉMICO,113001001719,INSTITUCION EDUCATIVA PROMOCION SOCIAL DE C/GENA.,S,URBANO,MAÑANA,13001,CARTAGENA DE INDIAS,13,BOLIVAR,N,13001,CARTAGENA DE INDIAS,BOLIVAR,13,66,91,4,57,70,3,41,29,2,74,99,4,64.0,89,A2,299

# Variables Globales

Empezamos con la definicion de una lista de columnas que luego usaremos para eliminar las columnas Inecesarias y ademas definimos una lista de columnas que por las cuales deberemos filtrar para evitar posibles valores atipicos

In [320]:
# lista de columnas que se eliminaran
drop_cols =[
]

# Arreglo con los filtros que se deben aplicar para eliminar datos atipicos
filters = {}


# Tamaño del dataset

In [321]:
len(df)

546212

## Análisis de Duplicados

In [322]:
df[df.duplicated(keep=False)]

,ESTU_TIPODOCUMENTO,ESTU_NACIONALIDAD,ESTU_GENERO,ESTU_FECHANACIMIENTO,PERIODO,ESTU_CONSECUTIVO,ESTU_ESTUDIANTE,ESTU_TIENEETNIA,ESTU_PAIS_RESIDE,ESTU_ETNIA,ESTU_DEPTO_RESIDE,ESTU_COD_RESIDE_DEPTO,ESTU_MCPIO_RESIDE,ESTU_COD_RESIDE_MCPIO,FAMI_ESTRATOVIVIENDA,FAMI_PERSONASHOGAR,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,FAMI_TRABAJOLABORPADRE,FAMI_TRABAJOLABORMADRE,FAMI_TIENEINTERNET,FAMI_TIENESERVICIOTV,FAMI_TIENECOMPUTADOR,FAMI_TIENELAVADORA,FAMI_TIENEHORNOMICROOGAS,FAMI_TIENEAUTOMOVIL,FAMI_TIENEMOTOCICLETA,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_NUMLIBROS,FAMI_COMELECHEDERIVADOS,FAMI_COMECARNEPESCADOHUEVO,FAMI_COMECEREALFRUTOSLEGUMBRE,FAMI_SITUACIONECONOMICA,ESTU_DEDICACIONLECTURADIARIA,ESTU_DEDICACIONINTERNET,ESTU_HORASSEMANATRABAJA,ESTU_TIPOREMUNERACION,COLE_CODIGO_ICFES,COLE_COD_DANE_ESTABLECIMIENTO,COLE_NOMBRE_ESTABLECIMIENTO,COLE_GENERO,COLE_NATURALEZA,COLE_CALENDARIO,COLE_BILINGUE,COLE_CARACTER,COLE_COD_DANE_SEDE,COLE_NOMBRE_SEDE,COLE_SEDE_PRINCIPAL,COLE_AREA_UBICACION,COLE_JORNADA,COLE_COD_MCPIO_UBICACION,COLE_MCPIO_UBICACION,COLE_COD_DEPTO_UBICACION,COLE_DEPTO_UBICACION,ESTU_PRIVADO_LIBERTAD,ESTU_COD_MCPIO_PRESENTACION,ESTU_MCPIO_PRESENTACION,ESTU_DEPTO_PRESENTACION,ESTU_COD_DEPTO_PRESENTACION,PUNT_LECTURA_CRITICA,PERCENTIL_LECTURA_CRITICA,DESEMP_LECTURA_CRITICA,PUNT_MATEMATICAS,PERCENTIL_MATEMATICAS,DESEMP_MATEMATICAS,PUNT_C_NATURALES,PERCENTIL_C_NATURALES,DESEMP_C_NATURALES,PUNT_SOCIALES_CIUDADANAS,PERCENTIL_SOCIALES_CIUDADANAS,DESEMP_SOCIALES_CIUDADANAS,PUNT_INGLES,PERCENTIL_INGLES,DESEMP_INGLES,PUNT_GLOBAL,PERCENTIL_GLOBAL,ESTU_INSE_INDIVIDUAL,ESTU_NSE_INDIVIDUAL,ESTU_NSE_ESTABLECIMIENTO,ESTU_ESTADOINVESTIGACION,ESTU_GENERACION-E


No hay filas duplicadas en el dataset

## Análisis de Nulos

In [323]:
df.isna().any(axis=1).sum()

56651

# Análisis de Columnas

## Columnas de texto

In [324]:

text_cols = df.select_dtypes(include = ["object", 'category']).columns.tolist() # listamos todas las columnas de texto
estu_text_cols = list(filter(lambda x: 'ESTU' in x, text_cols )) # Columnas con información del estudiante
fami_text_Cols = list(filter(lambda x: 'FAMI' in x, text_cols )) # Columnas con información de la familia del estudiante
cole_text_Cols = list(filter(lambda x: 'COLE' in x, text_cols )) # Columnas con información del colegio del estudiante
restante_texto_cols = list(filter(lambda x: not 'ESTU' in x and not 'FAMI' in x and not 'COLE' in x, text_cols)) # colummnas restantes

### Analizar columnas de texto de Estu

In [325]:
df[estu_text_cols].head()

,ESTU_TIPODOCUMENTO,ESTU_NACIONALIDAD,ESTU_GENERO,ESTU_FECHANACIMIENTO,ESTU_CONSECUTIVO,ESTU_ESTUDIANTE,ESTU_TIENEETNIA,ESTU_PAIS_RESIDE,ESTU_ETNIA,ESTU_DEPTO_RESIDE,ESTU_COD_RESIDE_DEPTO,ESTU_MCPIO_RESIDE,ESTU_COD_RESIDE_MCPIO,ESTU_DEDICACIONLECTURADIARIA,ESTU_DEDICACIONINTERNET,ESTU_HORASSEMANATRABAJA,ESTU_TIPOREMUNERACION,ESTU_PRIVADO_LIBERTAD,ESTU_MCPIO_PRESENTACION,ESTU_DEPTO_PRESENTACION,ESTU_INSE_INDIVIDUAL,ESTU_NSE_INDIVIDUAL,ESTU_ESTADOINVESTIGACION,ESTU_GENERACION-E
0,TI,COLOMBIA,M,02/18/2002 12:00:00 AM,SB11201940031558,ESTUDIANTE,No,COLOMBIA,-,MAGDALENA,47,SANTA ANA,47707,Entre 30 y 60 minutos,Entre 30 y 60 minutos,Menos de 10 horas,"Si, en efectivo",N,SANTA ANA,MAGDALENA,50.9421557404314,2,PUBLICAR,NO
1,TI,COLOMBIA,M,06/28/2003 12:00:00 AM,SB11201940303339,ESTUDIANTE,No,COLOMBIA,-,BOGOTÁ,11,BOGOTÁ D.C.,11001,Entre 30 y 60 minutos,Entre 30 y 60 minutos,Menos de 10 horas,"Si, en efectivo",N,BOGOTÁ D.C.,BOGOTÁ,59.767864431576,3,PUBLICAR,NO
2,TI,COLOMBIA,M,06/02/2001 12:00:00 AM,SB11201940255017,ESTUDIANTE,No,COLOMBIA,-,BOLIVAR,13,CARTAGENA DE INDIAS,13001,Entre 30 y 60 minutos,Más de 3 horas,0,"Si, en efectivo",N,CARTAGENA DE INDIAS,BOLIVAR,50.7642383644749,2,PUBLICAR,GENERACION E - GRATUIDAD
3,TI,COLOMBIA,M,04/26/2002 12:00:00 AM,SB11201940154799,ESTUDIANTE,Si,COLOMBIA,Ninguno,BOGOTÁ,11,BOGOTÁ D.C.,11001,30 minutos o menos,Entre 30 y 60 minutos,Más de 30 horas,"Si, en efectivo",N,BOGOTÁ D.C.,BOGOTÁ,45.7791353123516,2,PUBLICAR,GENERACION E - GRATUIDAD
4,TI,COLOMBIA,M,02/05/2002 12:00:00 AM,SB11201940280831,ESTUDIANTE,No,COLOMBIA,-,BOGOTÁ,11,BOGOTÁ D.C.,11001,No leo por entretenimiento,Más de 3 horas,Más de 30 horas,No,N,BOGOTÁ D.C.,BOGOTÁ,52.9109464277177,3,PUBLICAR,NO


In [326]:
df[estu_text_cols].describe(include='all')

,ESTU_TIPODOCUMENTO,ESTU_NACIONALIDAD,ESTU_GENERO,ESTU_FECHANACIMIENTO,ESTU_CONSECUTIVO,ESTU_ESTUDIANTE,ESTU_TIENEETNIA,ESTU_PAIS_RESIDE,ESTU_ETNIA,ESTU_DEPTO_RESIDE,ESTU_COD_RESIDE_DEPTO,ESTU_MCPIO_RESIDE,ESTU_COD_RESIDE_MCPIO,ESTU_DEDICACIONLECTURADIARIA,ESTU_DEDICACIONINTERNET,ESTU_HORASSEMANATRABAJA,ESTU_TIPOREMUNERACION,ESTU_PRIVADO_LIBERTAD,ESTU_MCPIO_PRESENTACION,ESTU_DEPTO_PRESENTACION,ESTU_INSE_INDIVIDUAL,ESTU_NSE_INDIVIDUAL,ESTU_ESTADOINVESTIGACION,ESTU_GENERACION-E
count,546212,546212,546212,546212,546212,546212,546212,546212,546212,545835,546212,545835,546212,545585,515914,545831,530704,546212,546212,546212,546208,529171,546212,546212
unique,10,33,3,11663,546212,1,3,33,21,34,35,1031,1115,6,6,6,5,2,472,33,469865,5,3,4
top,TI,COLOMBIA,F,01/01/1900 12:00:00 AM,SB11201940031558,ESTUDIANTE,No,COLOMBIA,-,BOGOTÁ,11,BOGOTÁ D.C.,11001,30 minutos o menos,Entre 1 y 3 horas,0,No,N,BOGOTÁ D.C.,BOGOTÁ,-,2,PUBLICAR,GENERACION E - GRATUIDAD
freq,445780,544918,295994,1826,1,546212,506074,544918,506203,83600,83600,83600,83600,199094,157557,354503,377436,546086,84122,84122,17041,197566,545755,297357


In [327]:
cols = ['ESTU_TIPODOCUMENTO', 'ESTU_NACIONALIDAD', 'ESTU_GENERO', 'ESTU_ESTUDIANTE', 'ESTU_TIENEETNIA', 'ESTU_PAIS_RESIDE', 'ESTU_ETNIA',
        'ESTU_DEPTO_RESIDE', 'ESTU_COD_RESIDE_DEPTO', 'ESTU_DEDICACIONLECTURADIARIA', 'ESTU_DEDICACIONINTERNET', 'ESTU_HORASSEMANATRABAJA',
       'ESTU_TIPOREMUNERACION', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_DEPTO_PRESENTACION', 'ESTU_NSE_INDIVIDUAL', 'ESTU_ESTADOINVESTIGACION', 
       'ESTU_GENERACION-E']

for col in cols:
    display(df[col].value_counts())
    print('-'*50)

ESTU_TIPODOCUMENTO
TI     445780
CC      91921
CR       6123
CE       1225
NES       822
PEP       235
PE         91
PC          6
RC          5
CCB         4
Name: count, dtype: int64

--------------------------------------------------


ESTU_NACIONALIDAD
COLOMBIA                544918
VENEZUELA                 1109
ESTADOS UNIDOS              52
ESPAÑA                      33
ECUADOR                     26
PERÚ                         7
CUBA                         6
MÉXICO                       6
BRASIL                       5
ITALIA                       5
BOLIVIA                      5
COSTA RICA                   4
ARGENTINA                    4
FRANCIA                      3
COMORAS                      3
                         ...  
CHINA                        2
REPÚBLICA DOMINICANA         2
COREA DEL NORTE              2
BÉLGICA                      1
PANAMÁ                       1
CURAZAO                      1
AFGANISTÁN                   1
REINO UNIDO                  1
DINAMARCA                    1
ARUBA                        1
ALEMANIA                     1
CROACIA                      1
TAIWAN                       1
ALBANIA                      1
VANUATU                      1
Name: count, Length: 

--------------------------------------------------


ESTU_GENERO
F    295994
M    250097
-       121
Name: count, dtype: int64

...

--------------------------------------------------


ESTU_ESTADOINVESTIGACION
PUBLICAR                                  545755
NO SE COMPROBO IDENTIDAD DEL EXAMINADO       272
VALIDEZ OFICINA JURÍDICA                     185
Name: count, dtype: int64

--------------------------------------------------


ESTU_GENERACION-E
GENERACION E - GRATUIDAD                   297357
NO                                         242966
GENERACION E - EXCELENCIA NACIONAL           5703
GENERACION E - EXCELENCIA DEPARTAMENTAL       186
Name: count, dtype: int64

--------------------------------------------------


In [328]:
drop_cols += [
    "ESTU_CONSECUTIVO", # Es un id que da el ICFES para presentar el exmanen, no es información propia del estudiante
    "ESTU_ESTUDIANTE", # Columna con un único valor ESTUDIANTE, no es información propia del estudiante
    "ESTU_COD_RESIDE_DEPTO", # Código del departamento, para este proyecto se aplicara one hot encode para departamentos, por esto no es necesario otro código
    "ESTU_MCPIO_RESIDE", # Se sale del alcance del proyecto, se busca predecir puntajes de estudiantes a nivel departamental
    "ESTU_COD_RESIDE_MCPIO", # Se sale del alcance del proyecto, se busca predecir puntajes de estudiantes a nivel departamental
    "ESTU_MCPIO_PRESENTACION", # Se sale del alcance del proyecto, se busca predecir puntajes de estudiantes a nivel departamental
]
"""
filters.update(
    {
        "ESTU_NACIONALIDAD": "COLOMBIA", # No tomamos las pruebas realizadas por extrangeros
        "ESTU_TIENEETNIA": "No",   # No tomamos a los estudiantes que especificaron su etnia dado que son muy pocos y quedaria desbalanceado
        "ESTU_PRIVADO_LIBERTAD": "N" # Los estudiantes privados de la libertad eran muy pocos y podrian representar valores atipicos
    }
)"""

'\nfilters.update(\n    {\n        "ESTU_NACIONALIDAD": "COLOMBIA", # No tomamos las pruebas realizadas por extrangeros\n        "ESTU_TIENEETNIA": "No",   # No tomamos a los estudiantes que especificaron su etnia dado que son muy pocos y quedaria desbalanceado\n        "ESTU_PRIVADO_LIBERTAD": "N" # Los estudiantes privados de la libertad eran muy pocos y podrian representar valores atipicos\n    }\n)'

### Analizar columnas de texto de Fami

In [329]:
df[fami_text_Cols].head()

,FAMI_ESTRATOVIVIENDA,FAMI_PERSONASHOGAR,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,FAMI_TRABAJOLABORPADRE,FAMI_TRABAJOLABORMADRE,FAMI_TIENEINTERNET,FAMI_TIENESERVICIOTV,FAMI_TIENECOMPUTADOR,FAMI_TIENELAVADORA,FAMI_TIENEHORNOMICROOGAS,FAMI_TIENEAUTOMOVIL,FAMI_TIENEMOTOCICLETA,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_NUMLIBROS,FAMI_COMELECHEDERIVADOS,FAMI_COMECARNEPESCADOHUEVO,FAMI_COMECEREALFRUTOSLEGUMBRE,FAMI_SITUACIONECONOMICA
0,Estrato 3,5 a 6,Cinco,Educación profesional completa,Secundaria (Bachillerato) completa,No aplica,"Trabaja en el hogar, no trabaja o estudia",Si,Si,No,Si,Si,No,No,No,MÁS DE 100 LIBROS,1 o 2 veces por semana,3 a 5 veces por semana,Nunca o rara vez comemos eso,Peor
1,Estrato 3,9 o más,Tres,Técnica o tecnológica completa,Técnica o tecnológica completa,Trabaja por cuenta propia (por ejemplo plomero...,Es dueño de un negocio pequeño (tiene pocos em...,Si,Si,Si,Si,Si,Si,No,No,MÁS DE 100 LIBROS,Todos o casi todos los días,3 a 5 veces por semana,1 o 2 veces por semana,Mejor
2,Estrato 1,5 a 6,Dos,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,Es vendedor o trabaja en atención al público,Es vendedor o trabaja en atención al público,No,Si,No,Si,No,No,No,No,26 A 100 LIBROS,Todos o casi todos los días,Todos o casi todos los días,3 a 5 veces por semana,Igual
3,Estrato 3,3 a 4,Dos,Primaria incompleta,Secundaria (Bachillerato) completa,No aplica,Es dueño de un negocio pequeño (tiene pocos em...,No,No,Si,No,No,No,No,No,11 A 25 LIBROS,Todos o casi todos los días,Todos o casi todos los días,3 a 5 veces por semana,Igual
4,Estrato 3,3 a 4,Cuatro,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) incompleta,"Trabaja en el hogar, no trabaja o estudia","Trabaja en el hogar, no trabaja o estudia",Si,Si,Si,Si,Si,No,No,No,26 A 100 LIBROS,Todos o casi todos los días,Nunca o rara vez comemos eso,Todos o casi todos los días,Mejor


In [330]:
df[fami_text_Cols].describe()

,FAMI_ESTRATOVIVIENDA,FAMI_PERSONASHOGAR,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,FAMI_TRABAJOLABORPADRE,FAMI_TRABAJOLABORMADRE,FAMI_TIENEINTERNET,FAMI_TIENESERVICIOTV,FAMI_TIENECOMPUTADOR,FAMI_TIENELAVADORA,FAMI_TIENEHORNOMICROOGAS,FAMI_TIENEAUTOMOVIL,FAMI_TIENEMOTOCICLETA,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_NUMLIBROS,FAMI_COMELECHEDERIVADOS,FAMI_COMECARNEPESCADOHUEVO,FAMI_COMECEREALFRUTOSLEGUMBRE,FAMI_SITUACIONECONOMICA
count,546186,537749,539794,537920,525253,538081,538743,537875,523932,538027,538569,537582,538006,537395,538487,537558,523907,536587,523880,537953
unique,8,6,7,13,13,14,14,3,3,3,3,3,3,3,3,5,5,5,5,4
top,Estrato 2,3 a 4,Tres,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,"Es agricultor, pesquero o jornalero","Trabaja en el hogar, no trabaja o estudia",Si,Si,Si,Si,No,No,No,No,0 A 10 LIBROS,1 o 2 veces por semana,Todos o casi todos los días,1 o 2 veces por semana,Igual
freq,188314,257604,207232,117488,136286,78187,213377,314042,391419,295409,405542,280967,401388,308054,410475,207526,170538,209228,198491,322524


In [331]:
for col in fami_text_Cols:
    display(df[col].value_counts())
    print('-'*50)

FAMI_ESTRATOVIVIENDA
Estrato 2      188314
Estrato 1      159977
Estrato 3      108692
-               34481
Estrato 4       25810
Sin Estrato     17177
Estrato 5        8024
Estrato 6        3711
Name: count, dtype: int64

--------------------------------------------------


FAMI_PERSONASHOGAR
3 a 4      257604
5 a 6      170028
7 a 8       44984
1 a 2       39843
9 o más     17760
-            7530
Name: count, dtype: int64

--------------------------------------------------


FAMI_CUARTOSHOGAR
Tres          207232
Dos           189042
Cuatro         70635
Uno            28276
Cinco          21966
Seis o mas     11903
-              10740
Name: count, dtype: int64

...

--------------------------------------------------


FAMI_COMECEREALFRUTOSLEGUMBRE
1 o 2 veces por semana          198491
3 a 5 veces por semana          154261
Todos o casi todos los días      88346
Nunca o rara vez comemos eso     71536
-                                11246
Name: count, dtype: int64

--------------------------------------------------


FAMI_SITUACIONECONOMICA
Igual    322524
Mejor    133690
Peor      72123
-          9616
Name: count, dtype: int64

--------------------------------------------------


### Analizar columnas de texto de Cole

In [332]:
df[cole_text_Cols].head()

,COLE_NOMBRE_ESTABLECIMIENTO,COLE_GENERO,COLE_NATURALEZA,COLE_CALENDARIO,COLE_BILINGUE,COLE_CARACTER,COLE_NOMBRE_SEDE,COLE_SEDE_PRINCIPAL,COLE_AREA_UBICACION,COLE_JORNADA,COLE_MCPIO_UBICACION,COLE_DEPTO_UBICACION
0,INSTITUCION EDUCATIVA DEPARTAMENTAL ANTONIO BR...,MIXTO,OFICIAL,A,N,TÉCNICO/ACADÉMICO,COL DPTAL ANTONIO BRUGES CARMONA,S,URBANO,MAÑANA,SANTA ANA,MAGDALENA
1,GIMN SAN ANGELO,MIXTO,NO OFICIAL,A,-,ACADÉMICO,GIMN SAN ANGELO,S,URBANO,COMPLETA,BOGOTÁ D.C.,BOGOTÁ
2,INSTITUCION EDUCATIVA PROMOCION SOCIAL DE C/GENA.,MIXTO,OFICIAL,A,N,TÉCNICO/ACADÉMICO,INSTITUCION EDUCATIVA PROMOCION SOCIAL DE C/GENA.,S,URBANO,MAÑANA,CARTAGENA DE INDIAS,BOLIVAR
3,COLEGIO VIRGINIA GUTIERREZ DE PINEDA (IED),MIXTO,OFICIAL,A,-,ACADÉMICO,COL VIRGINIA GUTIERREZ DE PINEDA (INS EDUC DIST),S,URBANO,MAÑANA,BOGOTÁ D.C.,BOGOTÁ
4,COLEGIO PALERMO IEDIP (IED),MIXTO,OFICIAL,A,N,TÉCNICO/ACADÉMICO,CENT EDUC DIST PALERMO CEDIP,S,URBANO,MAÑANA,BOGOTÁ D.C.,BOGOTÁ


In [333]:
df[cole_text_Cols].describe()

,COLE_NOMBRE_ESTABLECIMIENTO,COLE_GENERO,COLE_NATURALEZA,COLE_CALENDARIO,COLE_BILINGUE,COLE_CARACTER,COLE_NOMBRE_SEDE,COLE_SEDE_PRINCIPAL,COLE_AREA_UBICACION,COLE_JORNADA,COLE_MCPIO_UBICACION,COLE_DEPTO_UBICACION
count,546212,546212,546212,546212,546212,532464,546212,546212,546212,546212,546212,546212
unique,9372,3,2,3,3,5,10284,2,2,6,1030,33
top,INSTITUCION EDUCATIVA FRANCISCO JOSE DE CALDAS,MIXTO,OFICIAL,A,N,ACADÉMICO,INSTITUTO TECNICO INDUSTRIAL,S,URBANO,MAÑANA,BOGOTÁ D.C.,BOGOTÁ
freq,1147,528780,416011,541901,453280,284200,881,523322,460467,257060,82832,82832


In [334]:
cols = ['COLE_GENERO', 'COLE_NATURALEZA', 'COLE_CALENDARIO', 'COLE_BILINGUE', 'ESTU_TIENEETNIA', 'COLE_CARACTER', 'COLE_SEDE_PRINCIPAL',
        'COLE_AREA_UBICACION', 'COLE_JORNADA', 'COLE_DEPTO_UBICACION']

for col in cols:
    display(df[col].value_counts())
    print('-'*50)

COLE_GENERO
MIXTO        528780
FEMENINO      13462
MASCULINO      3970
Name: count, dtype: int64

--------------------------------------------------


COLE_NATURALEZA
OFICIAL       416011
NO OFICIAL    130201
Name: count, dtype: int64

--------------------------------------------------


COLE_CALENDARIO
A       541901
OTRO      3171
B         1140
Name: count, dtype: int64

...

--------------------------------------------------


COLE_JORNADA
MAÑANA      257060
COMPLETA     89520
UNICA        67361
TARDE        59049
SABATINA     39098
NOCHE        34124
Name: count, dtype: int64

--------------------------------------------------


COLE_DEPTO_UBICACION
BOGOTÁ             82832
ANTIOQUIA          74182
VALLE              38664
CUNDINAMARCA       37049
ATLANTICO          32235
SANTANDER          25751
BOLIVAR            25418
CORDOBA            19984
NARIÑO             16933
BOYACA             16737
MAGDALENA          16559
TOLIMA             16338
NORTE SANTANDER    15920
HUILA              14067
CAUCA              13972
                   ...  
RISARALDA          10737
CALDAS             10367
LA GUAJIRA          8835
QUINDIO             6460
CASANARE            5946
CHOCO               4874
CAQUETA             4669
PUTUMAYO            4037
ARAUCA              3339
GUAVIARE            1013
AMAZONAS             823
SAN ANDRES           620
VICHADA              497
VAUPES               352
GUAINIA              268
Name: count, Length: 33, dtype: int64

--------------------------------------------------


In [335]:

drop_cols+=[
    "COLE_MCPIO_UBICACION", # Se sale del alcance del proyecto, se busca predecir puntajes de estudiantes a nivel departamental
]
"""
filters.update(
    {
        "COLE_GENERO": "MIXTO",  # Esta variable categoria esta muy desvalanceada y escoger solo los colegios mixtos no afecta mucho el numero de registros
        "COLE_CALENDARIO": "A",   # Esta variable categoria esta muy desvalanceada y escoger solo los collegios con calendario A no afecta mucho el numero de registros
        "COLE_AREA_UBICACION": "URBANO", # Esta variable categoria esta muy desvalanceada y escoger solo los colegios urbanos no afecta mucho el numero de registros
        "COLE_NATURALEZA": "OFICIAL" #Esta variable categoria esta muy desvalanceada y escoger solo los colegios oficiales no afecta mucho el numero de registros
    }
)"""

'\nfilters.update(\n    {\n        "COLE_GENERO": "MIXTO",  # Esta variable categoria esta muy desvalanceada y escoger solo los colegios mixtos no afecta mucho el numero de registros\n        "COLE_CALENDARIO": "A",   # Esta variable categoria esta muy desvalanceada y escoger solo los collegios con calendario A no afecta mucho el numero de registros\n        "COLE_AREA_UBICACION": "URBANO", # Esta variable categoria esta muy desvalanceada y escoger solo los colegios urbanos no afecta mucho el numero de registros\n        "COLE_NATURALEZA": "OFICIAL" #Esta variable categoria esta muy desvalanceada y escoger solo los colegios oficiales no afecta mucho el numero de registros\n    }\n)'

### Analizar columnas de texto de Restantes

In [336]:
df[restante_texto_cols].head()

,DESEMP_INGLES,PERCENTIL_GLOBAL
0,A1,25
1,A2,83
2,A2,84
3,A1,88
4,A1,78


In [337]:
df[restante_texto_cols].describe()

,DESEMP_INGLES,PERCENTIL_GLOBAL
count,546212,546212
unique,5,101
top,A-,22
freq,271183,6443


In [338]:
"""
drop_cols+=[
    "DESEMP_INGLES", #No relevante para el objetivo
    "PERCENTIL_GLOBAL" #No relevante para el objetivo
]"""

'\ndrop_cols+=[\n    "DESEMP_INGLES", #No relevante para el objetivo\n    "PERCENTIL_GLOBAL" #No relevante para el objetivo\n]'

In [339]:
cols = ['DESEMP_INGLES', 'PERCENTIL_GLOBAL']

for col in cols:
    display(df[col].value_counts())

DESEMP_INGLES
A-    271183
A1    153405
A2     79450
B1     34853
B+      7321
Name: count, dtype: int64

PERCENTIL_GLOBAL
22    6443
69    6280
19    6183
3     6178
46    6111
44    6087
1     6073
17    6046
40    6017
51    6004
48    5997
43    5995
41    5957
81    5953
8     5950
      ... 
82    4954
93    4925
18    4827
73    4757
75    4608
45    4598
42    4541
47    4502
50    4493
52    4475
39    4405
36    4371
31    4320
56    4251
-       19
Name: count, Length: 101, dtype: int64

## Columnas Numericas

In [340]:
# listamos todas las columnas de numeros
num_cols=df.select_dtypes(include = ['float64','int32','int64']).columns.tolist()
estu_num_cols = list(filter(lambda x: 'ESTU' in x, num_cols))
fami_num_Cols = list(filter(lambda x: 'FAMI' in x, num_cols))
cole_num_Cols = list(filter(lambda x: 'COLE' in x, num_cols)) # Columnas con información del colegio del estudiante
restante_num_cols = list(filter(lambda x: not 'ESTU' in x and not 'FAMI' in x and not 'COLE' in x, num_cols))

### Analizar columnas de numericas de Estu

In [341]:
estu_num_cols

['ESTU_COD_MCPIO_PRESENTACION',
 'ESTU_COD_DEPTO_PRESENTACION',
 'ESTU_NSE_ESTABLECIMIENTO']

In [342]:
df[estu_num_cols].head()

,ESTU_COD_MCPIO_PRESENTACION,ESTU_COD_DEPTO_PRESENTACION,ESTU_NSE_ESTABLECIMIENTO
0,47707,47,2.0
1,11001,11,4.0
2,13001,13,3.0
3,11001,11,3.0
4,11001,11,3.0


In [343]:
drop_cols += [
        "ESTU_COD_MCPIO_PRESENTACION", # Se sale del alcance del proyecto, se busca predecir puntajes de estudiantes a nivel departamental
        "ESTU_COD_DEPTO_PRESENTACION", # Código del departamento, para este proyecto se aplicara one hot encode para departamentos, por esto no es necesario otro código

]

### Analizar columnas de numericas de Fami

In [344]:
fami_num_Cols

[]

No hay columnas de FAMI númericas

### Analizar columnas de numericas de Cole

In [345]:
cole_num_Cols

['COLE_CODIGO_ICFES',
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_COD_DANE_SEDE',
 'COLE_COD_MCPIO_UBICACION',
 'COLE_COD_DEPTO_UBICACION']

In [346]:
df[cole_num_Cols].head()

,COLE_CODIGO_ICFES,COLE_COD_DANE_ESTABLECIMIENTO,COLE_COD_DANE_SEDE,COLE_COD_MCPIO_UBICACION,COLE_COD_DEPTO_UBICACION
0,39008,147707000156,147707000156,47707,47
1,128611,311001105383,311001105383,11001,11
2,35493,113001001719,113001001719,13001,13
3,138032,111001107069,111001107069,11001,11
4,217406,111001094897,111001094897,11001,11


In [347]:
df[cole_num_Cols].astype(str).describe()

,COLE_CODIGO_ICFES,COLE_COD_DANE_ESTABLECIMIENTO,COLE_COD_DANE_SEDE,COLE_COD_MCPIO_UBICACION,COLE_COD_DEPTO_UBICACION
count,546212,546212,546212,546212,546212
unique,13297,10157,10939,1113,33
top,724211,105001000108,105001000108,11001,11
freq,869,869,869,82832,82832


Las columnas tienen tipo númerico, pero son códigos 

In [348]:
drop_cols += [
    "COLE_CODIGO_ICFES", # Código asignado por el ICFES, no es información propia del colegio
    "COLE_COD_DANE_ESTABLECIMIENTO", # Código asignado por el DANE, no es información propia del colegio
    "COLE_COD_DANE_SEDE", # Código asignado por el DANE, no es información propia del colegio
    "COLE_COD_MCPIO_UBICACION", # Se sale del alcance del proyecto, se busca predecir puntajes de estudiantes a nivel departamental
    "COLE_COD_DEPTO_UBICACION" # Código del departamento, para este proyecto se aplicara one hot encode para departamentos, por esto no es necesario otro código
]

### Analizar columnas de numericas Restantes

In [349]:
df[restante_num_cols].head()

,PERIODO,PUNT_LECTURA_CRITICA,PERCENTIL_LECTURA_CRITICA,DESEMP_LECTURA_CRITICA,PUNT_MATEMATICAS,PERCENTIL_MATEMATICAS,DESEMP_MATEMATICAS,PUNT_C_NATURALES,PERCENTIL_C_NATURALES,DESEMP_C_NATURALES,PUNT_SOCIALES_CIUDADANAS,PERCENTIL_SOCIALES_CIUDADANAS,DESEMP_SOCIALES_CIUDADANAS,PUNT_INGLES,PERCENTIL_INGLES,PUNT_GLOBAL
0,20194,47,33,2,48,42,2,37,15,1,30,8,1,54.0,67,208
1,20194,60,76,3,65,88,3,54,71,2,59,83,3,63.0,88,299
2,20194,66,91,4,57,70,3,41,29,2,74,99,4,64.0,89,299
3,20194,62,81,3,54,60,3,61,87,3,73,99,4,53.0,67,309
4,20194,63,85,3,57,70,3,55,74,2,57,79,3,52.0,63,288


In [350]:
df[restante_num_cols].describe()

,PERIODO,PUNT_LECTURA_CRITICA,PERCENTIL_LECTURA_CRITICA,DESEMP_LECTURA_CRITICA,PUNT_MATEMATICAS,PERCENTIL_MATEMATICAS,DESEMP_MATEMATICAS,PUNT_C_NATURALES,PERCENTIL_C_NATURALES,DESEMP_C_NATURALES,PUNT_SOCIALES_CIUDADANAS,PERCENTIL_SOCIALES_CIUDADANAS,DESEMP_SOCIALES_CIUDADANAS,PUNT_INGLES,PERCENTIL_INGLES,PUNT_GLOBAL
count,546212.0,546212.000000,546212.000000,546212.000000,546212.000000,546212.000000,546212.000000,546212.000000,546212.000000,546212.000000,546212.000000,546212.000000,546212.000000,546193.000000,546212.000000,546212.000000
mean,20194.0,52.157305,50.111151,2.612927,50.606349,50.110596,2.440651,48.234779,50.157646,2.020730,46.224589,50.179648,1.901584,48.416891,50.098669,246.186464
std,0.0,10.537963,28.959257,0.756910,11.997649,28.947324,0.752315,10.764035,28.935705,0.761262,12.140588,28.911682,0.820178,12.558438,28.917401,51.386858
min,20194.0,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000
25%,20194.0,45.000000,25.000000,2.000000,42.000000,25.000000,2.000000,40.000000,25.000000,1.000000,37.000000,25.000000,1.000000,39.000000,25.000000,207.000000
50%,20194.0,52.000000,50.000000,3.000000,51.000000,50.000000,3.000000,48.000000,50.000000,2.000000,45.000000,50.000000,2.000000,48.000000,50.000000,243.000000
75%,20194.0,60.000000,75.000000,3.000000,59.000000,75.000000,3.000000,56.000000,75.000000,3.000000,55.000000,75.000000,2.000000,56.000000,75.000000,282.000000
max,20194.0,100.000000,100.000000,4.000000,100.000000,100.000000,4.000000,100.000000,100.000000,4.000000,100.000000,100.000000,4.000000,100.000000,100.000000,477.000000


In [351]:
drop_cols+=[
    "PERIODO" # Columna con un solo valor que pone el ICFES, no esta relacionada con información del estudiante 
]

# Filtrado de Nulos

In [352]:
df_filtrado = df.dropna()

Se eliminan los valores "-" dado que en este dataset es otra forma de definir los nulos

In [353]:
df_filtrado = df_filtrado.drop(drop_cols, axis=1)
df_filtrado = df_filtrado.reindex(sorted(df_filtrado.columns), axis=1)
df_filtrado

,COLE_AREA_UBICACION,COLE_BILINGUE,COLE_CALENDARIO,COLE_CARACTER,COLE_DEPTO_UBICACION,COLE_GENERO,COLE_JORNADA,COLE_NATURALEZA,COLE_NOMBRE_ESTABLECIMIENTO,COLE_NOMBRE_SEDE,COLE_SEDE_PRINCIPAL,DESEMP_C_NATURALES,DESEMP_INGLES,DESEMP_LECTURA_CRITICA,DESEMP_MATEMATICAS,DESEMP_SOCIALES_CIUDADANAS,ESTU_DEDICACIONINTERNET,ESTU_DEDICACIONLECTURADIARIA,ESTU_DEPTO_PRESENTACION,ESTU_DEPTO_RESIDE,ESTU_ESTADOINVESTIGACION,ESTU_ETNIA,ESTU_FECHANACIMIENTO,ESTU_GENERACION-E,ESTU_GENERO,ESTU_HORASSEMANATRABAJA,ESTU_INSE_INDIVIDUAL,ESTU_NACIONALIDAD,ESTU_NSE_ESTABLECIMIENTO,ESTU_NSE_INDIVIDUAL,ESTU_PAIS_RESIDE,ESTU_PRIVADO_LIBERTAD,ESTU_TIENEETNIA,ESTU_TIPODOCUMENTO,ESTU_TIPOREMUNERACION,FAMI_COMECARNEPESCADOHUEVO,FAMI_COMECEREALFRUTOSLEGUMBRE,FAMI_COMELECHEDERIVADOS,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONMADRE,FAMI_EDUCACIONPADRE,FAMI_ESTRATOVIVIENDA,FAMI_NUMLIBROS,FAMI_PERSONASHOGAR,FAMI_SITUACIONECONOMICA,FAMI_TIENEAUTOMOVIL,FAMI_TIENECOMPUTADOR,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_TIENEHORNOMICROOGAS,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEMOTOCICLETA,FAMI_TIENESERVICIOTV,FAMI_TRABAJOLABORMADRE,FAMI_TRABAJOLABORPADRE,PERCENTIL_C_NATURALES,PERCENTIL_GLOBAL,PERCENTIL_INGLES,PERCENTIL_LECTURA_CRITICA,PERCENTIL_MATEMATICAS,PERCENTIL_SOCIALES_CIUDADANAS,PUNT_C_NATURALES,PUNT_GLOBAL,PUNT_INGLES,PUNT_LECTURA_CRITICA,PUNT_MATEMATICAS,PUNT_SOCIALES_CIUDADANAS
0,URBANO,N,A,TÉCNICO/ACADÉMICO,MAGDALENA,MIXTO,MAÑANA,OFICIAL,INSTITUCION EDUCATIVA DEPARTAMENTAL ANTONIO BR...,COL DPTAL ANTONIO BRUGES CARMONA,S,1,A1,2,2,1,Entre 30 y 60 minutos,Entre 30 y 60 minutos,MAGDALENA,MAGDALENA,PUBLICAR,-,02/18/2002 12:00:00 AM,NO,M,Menos de 10 horas,50.9421557404314,COLOMBIA,2.0,2,COLOMBIA,N,No,TI,"Si, en efectivo",3 a 5 veces por semana,Nunca o rara vez comemos eso,1 o 2 veces por semana,Cinco,Secundaria (Bachillerato) completa,Educación profesional completa,Estrato 3,MÁS DE 100 LIBROS,5 a 6,Peor,No,No,No,Si,Si,Si,No,Si,"Trabaja en el hogar, no trabaja o estudia",No aplica,15,25,67,33,42,8,37,208,54.0,47,48,30
1,URBANO,-,A,ACADÉMICO,BOGOTÁ,MIXTO,COMPLETA,NO OFICIAL,GIMN SAN ANGELO,GIMN SAN ANGELO,S,2,A2,3,3,3,Entre 30 y 60 minutos,Entre 30 y 60 minutos,BOGOTÁ,BOGOTÁ,PUBLICAR,-,06/28/2003 12:00:00 AM,NO,M,Menos de 10 horas,59.767864431576,COLOMBIA,4.0,3,COLOMBIA,N,No,TI,"Si, en efectivo",3 a 5 veces por semana,1 o 2 veces por semana,Todos o casi todos los días,Tres,Técnica o tecnológica completa,Técnica o tecnológica completa,Estrato 3,MÁS DE 100 LIBROS,9 o más,Mejor,Si,Si,No,Si,Si,Si,No,Si,Es dueño de un negocio pequeño (tiene pocos em...,Trabaja por cuenta propia (por ejemplo plomero...,71,83,88,76,88,83,54,299,63.0,60,65,59
2,URBANO,N,A,TÉCNICO/ACADÉMICO,BOLIVAR,MIXTO,MAÑANA,OFICIAL,INSTITUCION EDUCATIVA PROMOCION SOCIAL DE C/GENA.,INSTITUCION EDUCATIVA PROMOCION SOCIAL DE C/GENA.,S,2,A2,4,3,4,Más de 3 horas,Entre 30 y 60 minutos,BOLIVAR,BOLIVAR,PUBLICAR,-,06/02/2001 12:00:00 AM,GENERACION E - GRATUIDAD,M,0,50.7642383644749,COLOMBIA,3.0,2,COLOMBIA,N,No,TI,"Si, en efectivo",Todos o casi todos los días,3 a 5 veces por semana,Todos o casi todos los días,Dos,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,Estrato 1,26 A 100 LIBROS,5 a 6,Igual,No,No,No,No,No,Si,No,Si,Es vendedor o trabaja en atención al público,Es vendedor o trabaja en atención al público,29,84,89,91,70,99,41,299,64.0,66,57,74
3,URBANO,-,A,ACADÉMICO,BOGOTÁ,MIXTO,MAÑANA,OFICIAL,COLEGIO VIRGINIA GUTIERREZ DE PINEDA (IED),COL VIRGINIA GUTIERREZ DE PINEDA (INS EDUC DIST),S,3,A1,3,3,4,Entre 30 y 60 minutos,30 minutos o menos,BOGOTÁ,BOGOTÁ,PUBLICAR,Ninguno,04/26/2002 12:00:00 AM,GENERACION E - GRATUIDAD,M,Más de 30 horas,45.7791353123516,COLOMBIA,3.0,2,COLOMBIA,N,Si,TI,"Si, en efectivo",Todos o casi todos los días,3 a 5 veces por semana,Todos o casi todos los días,Dos,Secundaria (Bachillerato) completa,Primaria incompleta,Estrato 3,11 A 25 LIBROS,3 a 4,Igual,No,Si,No,No,No,No,No,No,Es dueño de un negocio pequeño (tiene pocos em...,No aplica,87,88,67,81,60,99,61,309,53.0,62,54,73
4,URBAN

# Imputación de Columnas

Hay texto con el caracter "-", esto significa que el campo no fue llenado. Descartar filas con "-" hace que el dataset se redusca en un 80%
Por lo que se imputaran utilizando el método de la categoría más frecuente

In [354]:
cols_with_character = list(df_filtrado.columns[df_filtrado.isin(['-']).any()])  # Se obtiene las columnas donde hay valores con -

In [355]:
df_filtrado[cols_with_character] = df_filtrado[cols_with_character].replace('-', np.nan)

In [356]:
imputer = CategoricalImputer(imputation_method='frequent')
df_filtrado[cols_with_character] = imputer.fit_transform(df_filtrado[cols_with_character])

In [357]:
df_filtrado[cols_with_character]

,COLE_BILINGUE,COLE_CARACTER,ESTU_DEDICACIONINTERNET,ESTU_DEDICACIONLECTURADIARIA,ESTU_ETNIA,ESTU_GENERO,ESTU_HORASSEMANATRABAJA,ESTU_TIPOREMUNERACION,FAMI_COMECARNEPESCADOHUEVO,FAMI_COMECEREALFRUTOSLEGUMBRE,FAMI_COMELECHEDERIVADOS,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONMADRE,FAMI_EDUCACIONPADRE,FAMI_ESTRATOVIVIENDA,FAMI_NUMLIBROS,FAMI_PERSONASHOGAR,FAMI_SITUACIONECONOMICA,FAMI_TIENEAUTOMOVIL,FAMI_TIENECOMPUTADOR,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_TIENEHORNOMICROOGAS,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEMOTOCICLETA,FAMI_TIENESERVICIOTV,FAMI_TRABAJOLABORMADRE,FAMI_TRABAJOLABORPADRE
0,N,TÉCNICO/ACADÉMICO,Entre 30 y 60 minutos,Entre 30 y 60 minutos,Comunidad afrodescendiente,M,Menos de 10 horas,"Si, en efectivo",3 a 5 veces por semana,Nunca o rara vez comemos eso,1 o 2 veces por semana,Cinco,Secundaria (Bachillerato) completa,Educación profesional completa,Estrato 3,MÁS DE 100 LIBROS,5 a 6,Peor,No,No,No,Si,Si,Si,No,Si,"Trabaja en el hogar, no trabaja o estudia",No aplica
1,N,ACADÉMICO,Entre 30 y 60 minutos,Entre 30 y 60 minutos,Comunidad afrodescendiente,M,Menos de 10 horas,"Si, en efectivo",3 a 5 veces por semana,1 o 2 veces por semana,Todos o casi todos los días,Tres,Técnica o tecnológica completa,Técnica o tecnológica completa,Estrato 3,MÁS DE 100 LIBROS,9 o más,Mejor,Si,Si,No,Si,Si,Si,No,Si,Es dueño de un negocio pequeño (tiene pocos em...,Trabaja por cuenta propia (por ejemplo plomero...
2,N,TÉCNICO/ACADÉMICO,Más de 3 horas,Entre 30 y 60 minutos,Comunidad afrodescendiente,M,0,"Si, en efectivo",Todos o casi todos los días,3 a 5 veces por semana,Todos o casi todos los días,Dos,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,Estrato 1,26 A 100 LIBROS,5 a 6,Igual,No,No,No,No,No,Si,No,Si,Es vendedor o trabaja en atención al público,Es vendedor o trabaja en atención al público
3,N,ACADÉMICO,Entre 30 y 60 minutos,30 minutos o menos,Ninguno,M,Más de 30 horas,"Si, en efectivo",Todos o casi todos los días,3 a 5 veces por semana,Todos o casi todos los días,Dos,Secundaria (Bachillerato) completa,Primaria incompleta,Estrato 3,11 A 25 LIBROS,3 a 4,Igual,No,Si,No,No,No,No,No,No,Es dueño de un negocio pequeño (tiene pocos em...,No aplica
4,N,TÉCNICO/ACADÉMICO,Más de 3 horas,No leo por entretenimiento,Comunidad afrodescendiente,M,Más de 30 horas,No,Nunca o rara vez comemos eso,Todos o casi todos los días,Todos o casi todos los días,Cuatro,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,Estrato 3,26 A 100 LIBROS,3 a 4,Mejor,No,Si,No,Si,Si,Si,No,Si,"Trabaja en el hogar, no trabaja o estudia","Trabaja en el hogar, no trabaja o estudia"
6,S,ACADÉMICO,Entre 30 y 60 minutos,30 minutos o menos,Comunidad afrodescendiente,M,0,No,Todos o casi todos los días,3 a 5 veces por semana,3 a 5 veces por semana,Tres,Postgrado,Técnica o tecnológica completa,Estrato 4,MÁS DE 100 LIBROS,3 a 4,Mejor,Si,Si,No,Si,Si,Si,No,Si,"Trabaja como profesional (por ejemplo médico, ...","Trabaja en el hogar, no trabaja o estudia"
7,N,TÉCNICO/ACADÉMICO,Más de 3 horas,Entre 1 y 2 horas,Comunidad afrodescendiente,M,Menos de 10 horas,"Si, en efectivo",3 a 5 veces por semana,3 a 5 veces por semana,1 o 2 veces por semana,Dos,Educación profesional completa,Educación profesional completa,Estrato 3,MÁS DE 100 LIBROS,3 a 4,Igual,No,Si,No,No,Si,Si,No,Si,"Trabaja en el hogar, no trabaja o estudia",Tiene un trabajo de tipo auxiliar administrati...
8,N,ACADÉMICO,Entre 1 y 3 horas,No leo por entretenimiento,Comunidad afrodescendiente,M,0,No,3 a 5 veces por semana,3 a 5 veces por semana,Todos o casi todos los días,Cuatro,Secundaria (Bachillerato) completa,No sabe,Estrato 3,26 A 100 LIBROS,5 a 6,Igual,No,Si,No,Si,Si,Si,No,Si,Tiene un trabajo de tipo auxiliar administrati...,No sabe
9,N,ACADÉMICO,Entre 30 y 60 minutos,Entre 30 y 60 minutos,Comunidad afrodescendiente,M,0,No,3 a 5 veces por semana,3 a 5 veces por semana,Todos o casi todos los días,Tres,Técnica o tecnológica completa,Secundaria (Bachillerato) completa,Estrato 3,11 A 25 LIBROS,5 a 6,

# Encoding

## Label Ordinary Encoding

Dado que varias de las columnas categoricas representan valores ordenados, el uso de One hot encoding nos representa una perdida de contexto importante

Ejemplo: la columna que nos indica el estrato de la casa donde reside el estudiante, [1,2,3,4,5,6] tiene un orden claro y nos puede dar informacion acerca de la situacion economica de la familia por lo cual hacer un encoding numerico ordenado, se ve como la mejor estrategia.

Esta funcion recibe el dataframe, la lista ordenada de categorias y el nombre de la columna a la cual se le aplicara el encoding

In [358]:
def orderEncode(df,categ,col):
    df2 = df
    for i in categ:
        df2[col] = df2[col].replace(categ,range(len(categ)))
    return df2[col]

In [359]:
df_encoded = df_filtrado

#------------ESTU_DEDICACIONINTERNET-----------------

categorias = [
    "No Navega Internet",
    "30 minutos o menos",
    "Entre 30 y 60 minutos",
    "Entre 1 y 3 horas",
    "Más de 3 horas"
]

df_encoded["ESTU_DEDICACIONINTERNET"] = orderEncode(df_encoded,categorias,"ESTU_DEDICACIONINTERNET")

#-------------ESTU_DEDICACIONLECTURADIARIA--------------


categorias = [
    "No leo por entretenimiento",
    "30 minutos o menos",
    "Entre 30 y 60 minutos",
    "Entre 1 y 2 horas",
    "Más de 2 horas"
]

df_encoded["ESTU_DEDICACIONLECTURADIARIA"] = orderEncode(df_encoded,categorias,"ESTU_DEDICACIONLECTURADIARIA")

#----------------ESTU_HORASSEMANATRABAJA--------------------

categorias = [
    '0',
    'Menos de 10 horas', 
    'Entre 11 y 20 horas',
    'Entre 21 y 30 horas',  
    'Más de 30 horas' 
]

df_encoded["ESTU_HORASSEMANATRABAJA"] = orderEncode(df_encoded,categorias,"ESTU_HORASSEMANATRABAJA")

#----------------FAMI_COMECARNEPESCADOHUEVO  &  FAMI_COMECEREALFRUTOSLEGUMBRE & FAMI_COMELECHEDERIVADOS--------------------

categorias = [
    'Nunca o rara vez comemos eso',
    '1 o 2 veces por semana',
    '3 a 5 veces por semana', 
    'Todos o casi todos los días'
]

df_encoded["FAMI_COMECARNEPESCADOHUEVO"] = orderEncode(df_encoded,categorias,"FAMI_COMECARNEPESCADOHUEVO")
df_encoded["FAMI_COMECEREALFRUTOSLEGUMBRE"] = orderEncode(df_encoded,categorias,"FAMI_COMECEREALFRUTOSLEGUMBRE")
df_encoded["FAMI_COMELECHEDERIVADOS"] = orderEncode(df_encoded,categorias,"FAMI_COMELECHEDERIVADOS")

#-----------------------FAMI_EDUCACIONMADRE & FAMI_EDUCACIONPADRE--------------------------------

categorias = [
    'No Aplica',
    'Ninguno',
    'No sabe',
    'Primaria incompleta',
    'Primaria completa',
    'Secundaria (Bachillerato) incompleta',
    'Secundaria (Bachillerato) completa',
    'Técnica o tecnológica incompleta',
    'Técnica o tecnológica completa',
    'Educación profesional incompleta', 
    'Educación profesional completa',
    'Postgrado'
]

df_encoded["FAMI_EDUCACIONMADRE"] = orderEncode(df_encoded,categorias,"FAMI_EDUCACIONMADRE")
df_encoded["FAMI_EDUCACIONPADRE"] = orderEncode(df_encoded,categorias,"FAMI_EDUCACIONPADRE")

#-------------------------------FAMI_ESTRATOVIVIENDA--------------------------

categorias = [
    'Sin Estrato', 
    'Estrato 1', 
    'Estrato 2',
    'Estrato 3', 
    'Estrato 4', 
    'Estrato 5',
    'Estrato 6'
]

df_encoded["FAMI_ESTRATOVIVIENDA"] = orderEncode(df_encoded,categorias,"FAMI_ESTRATOVIVIENDA")


#----------------------------------FAMI_NUMLIBROS--------------------------------

categorias = [
    '0 A 10 LIBROS',
    '11 A 25 LIBROS', 
    '26 A 100 LIBROS', 
    'MÁS DE 100 LIBROS'
]

df_encoded["FAMI_NUMLIBROS"] = orderEncode(df_encoded,categorias,"FAMI_NUMLIBROS")

#----------------------------------FAMI_PERSONASHOGAR--------------------------------

categorias = [
    '1 a 2',
    '3 a 4',
    '5 a 6',
    '7 a 8',
    '9 o más'
]

df_encoded["FAMI_PERSONASHOGAR"] = orderEncode(df_encoded,categorias,"FAMI_PERSONASHOGAR")

#----------------------------------FAMI_SITUACIONECONOMICA--------------------------------

categorias = [
    "Peor",
    "Igual",
    "Mejor"
]

df_encoded["FAMI_SITUACIONECONOMICA"] = orderEncode(df_encoded,categorias,"FAMI_SITUACIONECONOMICA")

#----------------------------------FAMI_TIENECOMPUTADOR & FAMI_TIENEINTERNET & FAMI_TIENESERVICIOTV & --------------------------------

categorias = [
    "No",
    "Si",
]

df_encoded["FAMI_TIENECOMPUTADOR"] = orderEncode(df_encoded,categorias,"FAMI_TIENECOMPUTADOR")
df_encoded["FAMI_TIENEINTERNET"] = orderEncode(df_encoded,categorias,"FAMI_TIENEINTERNET")
df_encoded["FAMI_TIENESERVICIOTV"] = orderEncode(df_encoded,categorias,"FAMI_TIENESERVICIOTV")

#----------------------------------ESTU_TIPOREMUNERACION--------------------------------
categorias = [
    "No",
    "Si, en especie",
    "Si, en efectivo y especie",
    "Si, en efectivo"
]

df_encoded["ESTU_TIPOREMUNERACION"] = orderEncode(df_encoded,categorias,"ESTU_TIPOREMUNERACION")

#------------------------------------------FAMI_TRABAJOLABORMADRE  &  FAMI_TRABAJOLABORPADRE-------------------------------

categorias = [
    'No sabe',
    'No aplica',
    'Trabaja en el hogar, no trabaja o estudia',
    'Trabaja como personal de limpieza, mantenimiento, seguridad o construcción',
    'Es agricultor, pesquero o jornalero', 
    'Es operario de máquinas o conduce vehículos (taxita, chofer)',
    'Es dueño de un negocio pequeño (tiene pocos empleados o no tiene, por ejemplo tienda, papelería, etc',
    'Es vendedor o trabaja en atención al público',
    'Tiene un trabajo de tipo auxiliar administrativo (por ejemplo, secretario o asistente)',
    'Trabaja por cuenta propia (por ejemplo plomero, electricista)',
    'Pensionado',
    'Trabaja como profesional (por ejemplo médico, abogado, ingeniero)',
    'Es dueño de un negocio grande, tiene un cargo de nivel directivo o gerencial'
]

df_encoded["FAMI_TRABAJOLABORMADRE"] = orderEncode(df_encoded,categorias,"FAMI_TRABAJOLABORMADRE")
df_encoded["FAMI_TRABAJOLABORPADRE"] = orderEncode(df_encoded,categorias,"FAMI_TRABAJOLABORPADRE")

#------------------------------------------ESTU_GENERO -------------------------------

categorias = [
    'M',
    'F'
]

df_encoded["ESTU_GENERO"] = orderEncode(df_encoded,categorias,"ESTU_GENERO")

#------------------------------------------COLE_BILINGUE-------------------------------

categorias = [
    'N',
    'S'
]

df_encoded["COLE_BILINGUE"] = orderEncode(df_encoded,categorias,"COLE_BILINGUE")

#------------------------------------------COLE_NATURALEZA-------------------------------

categorias = [
    'NO OFICIAL',
    'OFICIAL'
]

df_encoded["COLE_NATURALEZA"] = orderEncode(df_encoded,categorias,"COLE_NATURALEZA")

#------------------------------------------COLE_SEDE_PRINCIPAL-------------------------------

categorias = [
    'N',
    'S'
]

df_encoded["COLE_SEDE_PRINCIPAL"] = orderEncode(df_encoded,categorias,"COLE_SEDE_PRINCIPAL")

#------------------------------------------DESEMP_INGLES-------------------------------

categorias = [
    'A-',
    'A1',
    'A2',
    'B1',
    'B+'
]

df_encoded["DESEMP_INGLES"] = orderEncode(df_encoded,categorias,"DESEMP_INGLES")

#------------------------------------------ESTU_TIENEETNIA-------------------------------

categorias = [
    'No',
    'Si'
]

df_encoded["ESTU_TIENEETNIA"] = orderEncode(df_encoded,categorias,"ESTU_TIENEETNIA")


#------------------------------------------FAMI_CUARTOSHOGAR-------------------------------

categorias = [
    'Uno',
    'Dos',
    'Tres',
    'Cuatro',
    'Cinco',
    'Seis o mas'
]

df_encoded["FAMI_CUARTOSHOGAR"] = orderEncode(df_encoded,categorias,"FAMI_CUARTOSHOGAR")

#------------------------------------------FAMI_TIENEAUTOMOVIL-------------------------------
categorias = [
    'No',
    'Si'
]

df_encoded["FAMI_TIENEAUTOMOVIL"] = orderEncode(df_encoded,categorias,"FAMI_TIENEAUTOMOVIL")

#------------------------------------------FAMI_TIENECONSOLAVIDEOJUEGOS-------------------------------
categorias = [
    'No',
    'Si'
]

df_encoded["FAMI_TIENECONSOLAVIDEOJUEGOS"] = orderEncode(df_encoded,categorias,"FAMI_TIENECONSOLAVIDEOJUEGOS")

#------------------------------------------FAMI_TIENEHORNOMICROOGAS-------------------------------
categorias = [
    'No',
    'Si'
]

df_encoded["FAMI_TIENEHORNOMICROOGAS"] = orderEncode(df_encoded,categorias,"FAMI_TIENEHORNOMICROOGAS")

#------------------------------------------FAMI_TIENELAVADORA-------------------------------
categorias = [
    'No',
    'Si'
]

df_encoded["FAMI_TIENELAVADORA"] = orderEncode(df_encoded,categorias,"FAMI_TIENELAVADORA")

#------------------------------------------FAMI_TIENEMOTOCICLETA-------------------------------
categorias = [
    'No',
    'Si'
]

df_encoded["FAMI_TIENEMOTOCICLETA"] = orderEncode(df_encoded,categorias,"FAMI_TIENEMOTOCICLETA")

#------------------------------------------COLE_AREA_UBICACION-------------------------------
categorias = [
    'RURAL',
    'URBANO'
]

df_encoded["COLE_AREA_UBICACION"] = orderEncode(df_encoded,categorias,"COLE_AREA_UBICACION")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_25464\3421118362.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2[col] = df2[col].replace(categ,range(len(categ)))


## One-Hot Encoding

In [360]:
df_encoded = pd.get_dummies(df_encoded, columns = [
    'ESTU_DEPTO_RESIDE', 'COLE_CALENDARIO', 'COLE_CARACTER', 'COLE_DEPTO_UBICACION',
    'COLE_GENERO', 'COLE_JORNADA', 'ESTU_ESTADOINVESTIGACION', 'ESTU_ETNIA', 'ESTU_NACIONALIDAD', 'ESTU_PAIS_RESIDE', 
    'ESTU_PRIVADO_LIBERTAD', 'ESTU_TIPODOCUMENTO', 'ESTU_DEPTO_PRESENTACION', 'ESTU_GENERACION-E' ], dtype=int)

In [361]:
df_encoded

,COLE_AREA_UBICACION,COLE_BILINGUE,COLE_NATURALEZA,COLE_NOMBRE_ESTABLECIMIENTO,COLE_NOMBRE_SEDE,COLE_SEDE_PRINCIPAL,DESEMP_C_NATURALES,DESEMP_INGLES,DESEMP_LECTURA_CRITICA,DESEMP_MATEMATICAS,DESEMP_SOCIALES_CIUDADANAS,ESTU_DEDICACIONINTERNET,ESTU_DEDICACIONLECTURADIARIA,ESTU_FECHANACIMIENTO,ESTU_GENERO,ESTU_HORASSEMANATRABAJA,ESTU_INSE_INDIVIDUAL,ESTU_NSE_ESTABLECIMIENTO,ESTU_NSE_INDIVIDUAL,ESTU_TIENEETNIA,ESTU_TIPOREMUNERACION,FAMI_COMECARNEPESCADOHUEVO,FAMI_COMECEREALFRUTOSLEGUMBRE,FAMI_COMELECHEDERIVADOS,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONMADRE,FAMI_EDUCACIONPADRE,FAMI_ESTRATOVIVIENDA,FAMI_NUMLIBROS,FAMI_PERSONASHOGAR,FAMI_SITUACIONECONOMICA,FAMI_TIENEAUTOMOVIL,FAMI_TIENECOMPUTADOR,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_TIENEHORNOMICROOGAS,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEMOTOCICLETA,FAMI_TIENESERVICIOTV,FAMI_TRABAJOLABORMADRE,FAMI_TRABAJOLABORPADRE,PERCENTIL_C_NATURALES,PERCENTIL_GLOBAL,PERCENTIL_INGLES,PERCENTIL_LECTURA_CRITICA,PERCENTIL_MATEMATICAS,PERCENTIL_SOCIALES_CIUDADANAS,PUNT_C_NATURALES,PUNT_GLOBAL,PUNT_INGLES,PUNT_LECTURA_CRITICA,PUNT_MATEMATICAS,PUNT_SOCIALES_CIUDADANAS,ESTU_DEPTO_RESIDE_AMAZONAS,ESTU_DEPTO_RESIDE_ANTIOQUIA,ESTU_DEPTO_RESIDE_ARAUCA,ESTU_DEPTO_RESIDE_ATLANTICO,ESTU_DEPTO_RESIDE_BOGOTÁ,ESTU_DEPTO_RESIDE_BOLIVAR,ESTU_DEPTO_RESIDE_BOYACA,ESTU_DEPTO_RESIDE_CALDAS,ESTU_DEPTO_RESIDE_CAQUETA,ESTU_DEPTO_RESIDE_CASANARE,ESTU_DEPTO_RESIDE_CAUCA,ESTU_DEPTO_RESIDE_CESAR,ESTU_DEPTO_RESIDE_CHOCO,ESTU_DEPTO_RESIDE_CORDOBA,ESTU_DEPTO_RESIDE_CUNDINAMARCA,ESTU_DEPTO_RESIDE_EXTRANJERO,ESTU_DEPTO_RESIDE_GUAINIA,ESTU_DEPTO_RESIDE_GUAVIARE,ESTU_DEPTO_RESIDE_HUILA,ESTU_DEPTO_RESIDE_LA GUAJIRA,ESTU_DEPTO_RESIDE_MAGDALENA,ESTU_DEPTO_RESIDE_META,ESTU_DEPTO_RESIDE_NARIÑO,ESTU_DEPTO_RESIDE_NORTE SANTANDER,ESTU_DEPTO_RESIDE_PUTUMAYO,ESTU_DEPTO_RESIDE_QUINDIO,ESTU_DEPTO_RESIDE_RISARALDA,ESTU_DEPTO_RESIDE_SAN ANDRES,ESTU_DEPTO_RESIDE_SANTANDER,ESTU_DEPTO_RESIDE_SUCRE,ESTU_DEPTO_RESIDE_TOLIMA,ESTU_DEPTO_RESIDE_VALLE,ESTU_DEPTO_RESIDE_VAUPES,ESTU_DEPTO_RESIDE_VICHADA,COLE_CALENDARIO_A,COLE_CALENDARIO_B,COLE_CALENDARIO_OTRO,COLE_CARACTER_ACADÉMICO,COLE_CARACTER_NO APLICA,COLE_CARACTER_TÉCNICO,COLE_CARACTER_TÉCNICO/ACADÉMICO,COLE_DEPTO_UBICACION_AMAZONAS,COLE_DEPTO_UBICACION_ANTIOQUIA,COLE_DEPTO_UBICACION_ARAUCA,COLE_DEPTO_UBICACION_ATLANTICO,COLE_DEPTO_UBICACION_BOGOTÁ,COLE_DEPTO_UBICACION_BOLIVAR,COLE_DEPTO_UBICACION_BOYACA,COLE_DEPTO_UBICACION_CALDAS,COLE_DEPTO_UBICACION_CAQUETA,COLE_DEPTO_UBICACION_CASANARE,COLE_DEPTO_UBICACION_CAUCA,COLE_DEPTO_UBICACION_CESAR,COLE_DEPTO_UBICACION_CHOCO,COLE_DEPTO_UBICACION_CORDOBA,COLE_DEPTO_UBICACION_CUNDINAMARCA,COLE_DEPTO_UBICACION_GUAINIA,COLE_DEPTO_UBICACION_GUAVIARE,COLE_DEPTO_UBICACION_HUILA,COLE_DEPTO_UBICACION_LA GUAJIRA,COLE_DEPTO_UBICACION_MAGDALENA,COLE_DEPTO_UBICACION_META,COLE_DEPTO_UBICACION_NARIÑO,COLE_DEPTO_UBICACION_NORTE SANTANDER,COLE_DEPTO_UBICACION_PUTUMAYO,COLE_DEPTO_UBICACION_QUINDIO,COLE_DEPTO_UBICACION_RISARALDA,COLE_DEPTO_UBICACION_SAN ANDRES,COLE_DEPTO_UBICACION_SANTANDER,COLE_DEPTO_UBICACION_SUCRE,COLE_DEPTO_UBICACION_TOLIMA,COLE_DEPTO_UBICACION_VALLE,COLE_DEPTO_UBICACION_VAUPES,COLE_DEPTO_UBICACION_VICHADA,COLE_GENERO_FEMENINO,COLE_GENERO_MASCULINO,COLE_GENERO_MIXTO,COLE_JORNADA_COMPLETA,COLE_JORNADA_MAÑANA,COLE_JORNADA_NOCHE,COLE_JORNADA_SABATINA,COLE_JORNADA_TARDE,COLE_JORNADA_UNICA,ESTU_ESTADOINVESTIGACION_NO SE COMPROBO IDENTIDAD DEL EXAMINADO,ESTU_ESTADOINVESTIGACION_PUBLICAR,ESTU_ESTADOINVESTIGACION_VALIDEZ OFICINA JURÍDICA,ESTU_ETNIA_Arhuaco,ESTU_ETNIA_Cancuamo,ESTU_ETNIA_Comunidad afrodescendiente,ESTU_ETNIA_Comunidades Rom (Gitanas),ESTU_ETNIA_Cubeo,ESTU_ETNIA_Emberá,ESTU_ETNIA_Guambiano,ESTU_ETNIA_Huitoto,ESTU_ETNIA_Inga,ESTU_ETNIA_Ninguno,ESTU_ETNIA_Otro grupo étnico minoritario,ESTU_ETNIA_Paez,ESTU_ETNIA_Palenquero,ESTU_ETNIA_Pasto,ESTU_ETNIA_Pijao,ESTU_ETNIA_Raizal,ESTU_ETNIA_Sikuani,ESTU_ETNIA_Tucano,ESTU_ETNIA_Wayúu,ESTU_ETNIA_Zenú,ESTU_NACIONALIDAD_AFGANISTÁN,ESTU_NACIONALIDAD_ALBANIA,ESTU_NACIONALIDAD_ALEMANIA,ESTU_NACIONALIDAD_ARGENTINA,

## Label Encoding

In [362]:
label_encoder = preprocessing.LabelEncoder() 
df_encoded['COLE_NOMBRE_ESTABLECIMIENTO'] = label_encoder.fit_transform(df_encoded['COLE_NOMBRE_ESTABLECIMIENTO']) 
df_encoded['COLE_NOMBRE_SEDE'] = label_encoder.fit_transform(df_encoded['COLE_NOMBRE_SEDE'])   

## Encoding Fechas

### Descomposición de Fecha de Nacimiento

In [363]:
df_fecha_descompuesta = pd.DataFrame(data={'DIA_NACIMIENTO':[], 'MES_NACIMIENTO':[], 'ANIO_NACIMIENTO':[]})
df_fecha = df_encoded['ESTU_FECHANACIMIENTO'].apply(lambda x: x.split()[0])
df_fecha_descompuesta[['DIA_NACIMIENTO', 'MES_NACIMIENTO', 'ANIO_NACIMIENTO']] = df_fecha.str.split('/', expand=True)

Se reemplazan años que son incoherentes utilizando el método de máxima frecuencia, para este caso 2002.

In [364]:
df_fecha_descompuesta['ANIO_NACIMIENTO'] = df_fecha_descompuesta['ANIO_NACIMIENTO'].replace(['0001', '0002'], '2002')
df_fecha_descompuesta = df_fecha_descompuesta.astype(int)
df_encoded = df_encoded.drop(['ESTU_FECHANACIMIENTO'], axis=1, errors='ignore')
df_encoded[['DIA_NACIMIENTO', 'MES_NACIMIENTO', 'ANIO_NACIMIENTO']] = df_fecha_descompuesta[['DIA_NACIMIENTO', 'MES_NACIMIENTO', 'ANIO_NACIMIENTO']]

Se aplica el encode ciclico para día y mes de nacimiento

In [365]:
cyclical = CyclicalFeatures(variables=None, drop_original=True)

df_encoded[['DIA_NACIMIENTO_sin', 'DIA_NACIMIENTO_cos', 'MES_NACIMIENTO_sin', 'MES_NACIMIENTO_cos']]  = cyclical.fit_transform(df_encoded[['DIA_NACIMIENTO', 'MES_NACIMIENTO']])
df_encoded = df_encoded.drop([['DIA_NACIMIENTO', 'MES_NACIMIENTO']], axis=1, errors='ignore')
df_encoded

,COLE_AREA_UBICACION,COLE_BILINGUE,COLE_NATURALEZA,COLE_NOMBRE_ESTABLECIMIENTO,COLE_NOMBRE_SEDE,COLE_SEDE_PRINCIPAL,DESEMP_C_NATURALES,DESEMP_INGLES,DESEMP_LECTURA_CRITICA,DESEMP_MATEMATICAS,DESEMP_SOCIALES_CIUDADANAS,ESTU_DEDICACIONINTERNET,ESTU_DEDICACIONLECTURADIARIA,ESTU_GENERO,ESTU_HORASSEMANATRABAJA,ESTU_INSE_INDIVIDUAL,ESTU_NSE_ESTABLECIMIENTO,ESTU_NSE_INDIVIDUAL,ESTU_TIENEETNIA,ESTU_TIPOREMUNERACION,FAMI_COMECARNEPESCADOHUEVO,FAMI_COMECEREALFRUTOSLEGUMBRE,FAMI_COMELECHEDERIVADOS,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONMADRE,FAMI_EDUCACIONPADRE,FAMI_ESTRATOVIVIENDA,FAMI_NUMLIBROS,FAMI_PERSONASHOGAR,FAMI_SITUACIONECONOMICA,FAMI_TIENEAUTOMOVIL,FAMI_TIENECOMPUTADOR,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_TIENEHORNOMICROOGAS,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEMOTOCICLETA,FAMI_TIENESERVICIOTV,FAMI_TRABAJOLABORMADRE,FAMI_TRABAJOLABORPADRE,PERCENTIL_C_NATURALES,PERCENTIL_GLOBAL,PERCENTIL_INGLES,PERCENTIL_LECTURA_CRITICA,PERCENTIL_MATEMATICAS,PERCENTIL_SOCIALES_CIUDADANAS,PUNT_C_NATURALES,PUNT_GLOBAL,PUNT_INGLES,PUNT_LECTURA_CRITICA,PUNT_MATEMATICAS,PUNT_SOCIALES_CIUDADANAS,ESTU_DEPTO_RESIDE_AMAZONAS,ESTU_DEPTO_RESIDE_ANTIOQUIA,ESTU_DEPTO_RESIDE_ARAUCA,ESTU_DEPTO_RESIDE_ATLANTICO,ESTU_DEPTO_RESIDE_BOGOTÁ,ESTU_DEPTO_RESIDE_BOLIVAR,ESTU_DEPTO_RESIDE_BOYACA,ESTU_DEPTO_RESIDE_CALDAS,ESTU_DEPTO_RESIDE_CAQUETA,ESTU_DEPTO_RESIDE_CASANARE,ESTU_DEPTO_RESIDE_CAUCA,ESTU_DEPTO_RESIDE_CESAR,ESTU_DEPTO_RESIDE_CHOCO,ESTU_DEPTO_RESIDE_CORDOBA,ESTU_DEPTO_RESIDE_CUNDINAMARCA,ESTU_DEPTO_RESIDE_EXTRANJERO,ESTU_DEPTO_RESIDE_GUAINIA,ESTU_DEPTO_RESIDE_GUAVIARE,ESTU_DEPTO_RESIDE_HUILA,ESTU_DEPTO_RESIDE_LA GUAJIRA,ESTU_DEPTO_RESIDE_MAGDALENA,ESTU_DEPTO_RESIDE_META,ESTU_DEPTO_RESIDE_NARIÑO,ESTU_DEPTO_RESIDE_NORTE SANTANDER,ESTU_DEPTO_RESIDE_PUTUMAYO,ESTU_DEPTO_RESIDE_QUINDIO,ESTU_DEPTO_RESIDE_RISARALDA,ESTU_DEPTO_RESIDE_SAN ANDRES,ESTU_DEPTO_RESIDE_SANTANDER,ESTU_DEPTO_RESIDE_SUCRE,ESTU_DEPTO_RESIDE_TOLIMA,ESTU_DEPTO_RESIDE_VALLE,ESTU_DEPTO_RESIDE_VAUPES,ESTU_DEPTO_RESIDE_VICHADA,COLE_CALENDARIO_A,COLE_CALENDARIO_B,COLE_CALENDARIO_OTRO,COLE_CARACTER_ACADÉMICO,COLE_CARACTER_NO APLICA,COLE_CARACTER_TÉCNICO,COLE_CARACTER_TÉCNICO/ACADÉMICO,COLE_DEPTO_UBICACION_AMAZONAS,COLE_DEPTO_UBICACION_ANTIOQUIA,COLE_DEPTO_UBICACION_ARAUCA,COLE_DEPTO_UBICACION_ATLANTICO,COLE_DEPTO_UBICACION_BOGOTÁ,COLE_DEPTO_UBICACION_BOLIVAR,COLE_DEPTO_UBICACION_BOYACA,COLE_DEPTO_UBICACION_CALDAS,COLE_DEPTO_UBICACION_CAQUETA,COLE_DEPTO_UBICACION_CASANARE,COLE_DEPTO_UBICACION_CAUCA,COLE_DEPTO_UBICACION_CESAR,COLE_DEPTO_UBICACION_CHOCO,COLE_DEPTO_UBICACION_CORDOBA,COLE_DEPTO_UBICACION_CUNDINAMARCA,COLE_DEPTO_UBICACION_GUAINIA,COLE_DEPTO_UBICACION_GUAVIARE,COLE_DEPTO_UBICACION_HUILA,COLE_DEPTO_UBICACION_LA GUAJIRA,COLE_DEPTO_UBICACION_MAGDALENA,COLE_DEPTO_UBICACION_META,COLE_DEPTO_UBICACION_NARIÑO,COLE_DEPTO_UBICACION_NORTE SANTANDER,COLE_DEPTO_UBICACION_PUTUMAYO,COLE_DEPTO_UBICACION_QUINDIO,COLE_DEPTO_UBICACION_RISARALDA,COLE_DEPTO_UBICACION_SAN ANDRES,COLE_DEPTO_UBICACION_SANTANDER,COLE_DEPTO_UBICACION_SUCRE,COLE_DEPTO_UBICACION_TOLIMA,COLE_DEPTO_UBICACION_VALLE,COLE_DEPTO_UBICACION_VAUPES,COLE_DEPTO_UBICACION_VICHADA,COLE_GENERO_FEMENINO,COLE_GENERO_MASCULINO,COLE_GENERO_MIXTO,COLE_JORNADA_COMPLETA,COLE_JORNADA_MAÑANA,COLE_JORNADA_NOCHE,COLE_JORNADA_SABATINA,COLE_JORNADA_TARDE,COLE_JORNADA_UNICA,ESTU_ESTADOINVESTIGACION_NO SE COMPROBO IDENTIDAD DEL EXAMINADO,ESTU_ESTADOINVESTIGACION_PUBLICAR,ESTU_ESTADOINVESTIGACION_VALIDEZ OFICINA JURÍDICA,ESTU_ETNIA_Arhuaco,ESTU_ETNIA_Cancuamo,ESTU_ETNIA_Comunidad afrodescendiente,ESTU_ETNIA_Comunidades Rom (Gitanas),ESTU_ETNIA_Cubeo,ESTU_ETNIA_Emberá,ESTU_ETNIA_Guambiano,ESTU_ETNIA_Huitoto,ESTU_ETNIA_Inga,ESTU_ETNIA_Ninguno,ESTU_ETNIA_Otro grupo étnico minoritario,ESTU_ETNIA_Paez,ESTU_ETNIA_Palenquero,ESTU_ETNIA_Pasto,ESTU_ETNIA_Pijao,ESTU_ETNIA_Raizal,ESTU_ETNIA_Sikuani,ESTU_ETNIA_Tucano,ESTU_ETNIA_Wayúu,ESTU_ETNIA_Zenú,ESTU_NACIONALIDAD_AFGANISTÁN,ESTU_NACIONALIDAD_ALBANIA,ESTU_NACIONALIDAD_ALEMANIA,ESTU_NACIONALIDAD_ARGENTINA,ESTU_NACIONALIDAD_ARU

In [366]:
"""cols = df_encoded.columns.tolist()
target_cols = list(filter(lambda x: 'PUNT' in x, cols )) # Columnas TARGET
input_cols = list(set(cols).difference(set(target_cols)))
estu_cols = list(filter(lambda x: 'ESTU' in x, input_cols )) 
fami_cols = list(filter(lambda x: 'FAMI' in x, input_cols ))
resid_cols = list(filter(lambda x: 'ESTU_DEPTO_RESIDE' in x, input_cols ))
estu_sin_resi_cols = list(set(estu_cols).difference(set(resid_cols)))"""

"cols = df_encoded.columns.tolist()\ntarget_cols = list(filter(lambda x: 'PUNT' in x, cols )) # Columnas TARGET\ninput_cols = list(set(cols).difference(set(target_cols)))\nestu_cols = list(filter(lambda x: 'ESTU' in x, input_cols )) \nfami_cols = list(filter(lambda x: 'FAMI' in x, input_cols ))\nresid_cols = list(filter(lambda x: 'ESTU_DEPTO_RESIDE' in x, input_cols ))\nestu_sin_resi_cols = list(set(estu_cols).difference(set(resid_cols)))"

# Analisis avanzado

In [367]:
#a = df_encoded[estu_sin_resi_cols+fami_cols].hist(figsize=(18,15))

# Dataset

Se crean los datasets de train y test, teniando como Y todas las columnas de resultados de las diferentes areas y como X los datos del estudiante y su familia

In [368]:
# Creación de datasets
#X_train, X_test, y_train, y_test = train_test_split(df_encoded[input_cols], df_encoded[target_cols], test_size=0.3, random_state=42)

In [369]:
#X_train.columns

In [370]:
#y_train.columns